In [137]:
# Imports
import pandas as pd
import os
import numpy as np

In [ ]:
# Define the directory path
directory_path = '../../data/intermediate'

# List all files in the directory
files = [f for f in os.listdir(directory_path) if f.endswith('.csv') and f != 'data_dic.csv' and f != 'model_table.csv']

# Define the correct datatypes for pandas
dtypes = {
    'HCP_ID': 'str',
    'PATIENT_AGE': 'Int64',
    'PATIENT_GENDER': 'category',
    'PATIENT_ID': 'str',
    'STATE': 'category',
    'HCP_SPECIALTY': 'category',
    'HCP_AGE': 'Int64',
    'HCP_GENDER': 'category',
    'STATE_NAME': 'category',
    'TXN_LOCATION_TYPE': 'category',
    'INSURANCE_TYPE': 'category',
    'TXN_TYPE': 'category',
    'TXN_DESC': 'category'
}

# Specify the datetime columns
parse_dates = ['TXN_DT', 'BIRTH_DT']

# Load the CSV files into a dictionary of dataframes
dataframes = {}
for file in files:
    file_path = os.path.join(directory_path, file)
    
    # Read the CSV file without specifying parse_dates
    df = pd.read_csv(file_path, dtype=dtypes)
    
    # Check if 'TXN_DT' column exists and parse it as datetime if it does
    if 'TXN_DT' in df.columns:
        df['TXN_DT'] = pd.to_datetime(df['TXN_DT'])
    
    dataframes[file] = df

# Print the loaded dataframes
for file, df in dataframes.items():
    print(f"Data from {file}:")
    print(df.head())

Data from dim_physician.csv:
  HCP_ID STATE        HCP_SPECIALTY  HCP_AGE HCP_GENDER  STATE_NAME
0      1    TX     PHYSICAL THERAPY     <NA>  U-Unknown       Texas
1      2    IN  PHYSICIAN ASSISTANT     <NA>  U-Unknown     Indiana
2      3    CA   EMERGENCY MEDICINE     <NA>  U-Unknown  California
3      4    TX   NURSE PRACTITIONER     <NA>  U-Unknown       Texas
4      5    WA   EMERGENCY MEDICINE     <NA>  U-Unknown  Washington
Data from fact_txn.csv:
      TXN_DT PATIENT_ID HCP_ID       TXN_LOCATION_TYPE INSURANCE_TYPE  \
0 2021-12-27       3124    NaN  INDEPENDENT LABORATORY     COMMERCIAL   
1 2020-02-25       3352  21972                  OFFICE     COMMERCIAL   
2 2022-01-25       2902  21586     HOSPITAL OUTPATIENT       MEDICARE   
3 2022-03-05       2943   2439              UNASSIGNED     COMMERCIAL   
4 2018-01-03       3589   6381                  OFFICE       MEDICARE   

     TXN_TYPE           TXN_DESC  
0    SYMPTOMS  ACUTE_PHARYNGITIS  
1  CONDITIONS            OBESI

In [139]:
# Create a dataframe from the dataframes dictionary
dim_patient = dataframes['dim_patient.csv']
dim_physician = dataframes['dim_physician.csv']
fact_txn = dataframes['fact_txn.csv']

In [140]:
# Calculate num conditions from fact_txn table

fact_txn_num_conditions = fact_txn.groupby('PATIENT_ID')['TXN_TYPE'].apply(lambda x: (x == 'CONDITIONS').sum()).reset_index(name='NUM_CONDITIONS')
print(fact_txn_num_conditions)

     PATIENT_ID  NUM_CONDITIONS
0             1               1
1            10               1
2           100               1
3          1000               5
4          1001              20
...         ...             ...
4015        995               1
4016        996               2
4017        997              18
4018        998               3
4019        999              24

[4020 rows x 2 columns]


In [141]:
# Testing
display(fact_txn_num_conditions[fact_txn_num_conditions['PATIENT_ID'] == '4017'])

,PATIENT_ID,NUM_CONDITIONS
3354,4017,64


In [142]:
display(fact_txn_num_conditions[fact_txn_num_conditions['PATIENT_ID'] == '3476'])

,PATIENT_ID,NUM_CONDITIONS
2752,3476,9


In [143]:
# Calculate num contraindications from fact_txn table

fact_txn_num_contraindications = fact_txn.groupby('PATIENT_ID')['TXN_TYPE'].apply(lambda x: (x == 'CONTRAINDICATIONS').sum()).reset_index(name='NUM_CONTRAINDICATIONS')
fact_txn_num_contraindications

,PATIENT_ID,NUM_CONTRAINDICATIONS
0,1,0
1,10,0
2,100,0
3,1000,0
4,1001,3
...,...,...
4015,995,0
4016,996,0
4017,997,0
4018,998,0


In [144]:
# Calculate num symptoms from fact_txn table

fact_txn_num_symptoms = fact_txn.groupby('PATIENT_ID')['TXN_TYPE'].apply(lambda x: (x == 'SYMPTOMS').sum()).reset_index(name='NUM_SYMPTOMS')
print(fact_txn_num_symptoms)

     PATIENT_ID  NUM_SYMPTOMS
0             1             0
1            10             0
2           100             1
3          1000            16
4          1001             5
...         ...           ...
4015        995             3
4016        996            13
4017        997            12
4018        998             6
4019        999            18

[4020 rows x 2 columns]


In [145]:
# Calculate target (Drug A patients)
fact_txn_treatment= fact_txn.groupby('PATIENT_ID')['TXN_DESC'].apply(lambda x: (x == 'DRUG A').sum()).reset_index(name='DRUGA_COUNT')
fact_txn_treatment.head(100)

,PATIENT_ID,DRUGA_COUNT
0,1,0
1,10,0
2,100,0
3,1000,0
4,1001,0
...,...,...
95,1084,0
96,1085,0
97,1086,0
98,1087,0


In [146]:
display(fact_txn_treatment[fact_txn_treatment['PATIENT_ID'] == '4010'])

,PATIENT_ID,DRUGA_COUNT
3347,4010,1


In [147]:
# Patients that were treated with drug A more than once
fact_txn_treatment.loc[fact_txn_treatment['DRUGA_COUNT'] > 1]

,PATIENT_ID,DRUGA_COUNT
651,1585,2
1119,2005,2
1165,2047,2
1875,2687,2
3246,3920,2
3324,3991,2
3690,701,2


In [148]:
# Now create the boolean (1,0) target column
fact_txn_treatment['TARGET'] = np.where(fact_txn_treatment['DRUGA_COUNT'] >= 1, 1, 0)
fact_txn_treatment.head()

,PATIENT_ID,DRUGA_COUNT,TARGET
0,1,0,0
1,10,0,0
2,100,0,0
3,1000,0,0
4,1001,0,0


In [149]:
display(fact_txn_treatment[fact_txn_treatment['PATIENT_ID'] == '701'])

,PATIENT_ID,DRUGA_COUNT,TARGET
3690,701,2,1


In [150]:
# Patients with prescribed with drug a more than once 
fact_txn_treatment.loc[fact_txn_treatment['DRUGA_COUNT'] > 1]

,PATIENT_ID,DRUGA_COUNT,TARGET
651,1585,2,1
1119,2005,2,1
1165,2047,2,1
1875,2687,2,1
3246,3920,2,1
3324,3991,2,1
3690,701,2,1


In [151]:
#disease x diagnosis fact_txn

#create a df grouped by patient ID that counts how many time a patient has been diagnosed with disease x
fact_txn_disease_x = fact_txn.groupby('PATIENT_ID')['TXN_DESC'].apply(lambda x: (x == 'DISEASE_X').sum()).reset_index(name='DISEASEX_COUNT')
print(fact_txn_disease_x)

     PATIENT_ID  DISEASEX_COUNT
0             1               1
1            10               1
2           100               1
3          1000               2
4          1001               1
...         ...             ...
4015        995               1
4016        996               1
4017        997               2
4018        998               1
4019        999               3

[4020 rows x 2 columns]


In [152]:
fact_txn_disease_x[fact_txn_disease_x['PATIENT_ID'] == '401']

,PATIENT_ID,DISEASEX_COUNT
3346,401,22


In [153]:
# Patients diagnosed with disease x more than once
fact_txn_disease_x.loc[fact_txn_disease_x['DISEASEX_COUNT'] > 1]

,PATIENT_ID,DISEASEX_COUNT
3,1000,2
6,1003,2
7,1004,2
8,1005,2
9,1006,4
...,...,...
4004,985,2
4008,989,2
4009,99,3
4017,997,2


In [154]:
#features from first time disease x was diagnosed need to be extracted 

#sort values by date
fact_txn_disease_x_features = fact_txn.sort_values(by='TXN_DT',ascending=True)

#filter by txn_desc == disease x
fact_txn_disease_x_features = fact_txn_disease_x_features[fact_txn_disease_x_features['TXN_DESC'] == 'DISEASE_X']
fact_txn_disease_x_features

,TXN_DT,PATIENT_ID,HCP_ID,TXN_LOCATION_TYPE,INSURANCE_TYPE,TXN_TYPE,TXN_DESC
9199,2022-04-01,683,16981,UNASSIGNED,MEDICAID,CONDITIONS,DISEASE_X
105575,2022-04-03,1176,NaN,INDEPENDENT LABORATORY,COMMERCIAL,CONDITIONS,DISEASE_X
87097,2022-04-08,2295,NaN,OTHER PLACE OF SERVICE,COMMERCIAL,CONDITIONS,DISEASE_X
94077,2022-04-08,2509,NaN,OTHER PLACE OF SERVICE,COMMERCIAL,CONDITIONS,DISEASE_X
65429,2022-04-09,3208,18291,URGENT CARE FACILITY,MEDICARE,CONDITIONS,DISEASE_X
...,...,...,...,...,...,...,...
40241,2022-07-22,3604,14536,OFFICE,COMMERCIAL,CONDITIONS,DISEASE_X
60809,2022-07-22,1019,15987,INPATIENT HOSPITAL,COMMERCIAL,CONDITIONS,DISEASE_X
73663,2022-07-22,1590,20531,UNASSIGNED,COMMERCIAL,CONDITIONS,DISEASE_X
105428,2022-07-22,85,16215,INPATIENT HOSPITAL,MEDICARE,CONDITIONS,DISEASE_X


In [155]:
#Create a rank (cumulative sum) of disease x diagnosis per patient, as it is sorted by date rank == 1 shall be the first diagnosis for each patient
fact_txn_disease_x_features['RANK'] = fact_txn_disease_x_features.groupby('PATIENT_ID').cumcount() + 1
fact_txn_disease_x_features.head(30)

,TXN_DT,PATIENT_ID,HCP_ID,TXN_LOCATION_TYPE,INSURANCE_TYPE,TXN_TYPE,TXN_DESC,RANK
9199,2022-04-01,683,16981,UNASSIGNED,MEDICAID,CONDITIONS,DISEASE_X,1
105575,2022-04-03,1176,NaN,INDEPENDENT LABORATORY,COMMERCIAL,CONDITIONS,DISEASE_X,1
87097,2022-04-08,2295,NaN,OTHER PLACE OF SERVICE,COMMERCIAL,CONDITIONS,DISEASE_X,1
94077,2022-04-08,2509,NaN,OTHER PLACE OF SERVICE,COMMERCIAL,CONDITIONS,DISEASE_X,1
65429,2022-04-09,3208,18291,URGENT CARE FACILITY,MEDICARE,CONDITIONS,DISEASE_X,1
10639,2022-04-10,3208,NaN,INDEPENDENT LABORATORY,MEDICARE,CONDITIONS,DISEASE_X,2
29216,2022-04-11,3208,15000,URGENT CARE FACILITY,MEDICARE,CONDITIONS,DISEASE_X,3
45121,2022-04-11,3208,1976,URGENT CARE FACILITY,MEDICARE,CONDITIONS,DISEASE_X,4
95936,2022-04-11,189,NaN,OTHER PLACE OF SERVICE,MEDICARE,CONDITIONS,DISEASE_X,1
46362,2022-04-12,401,678,HOSPITAL INPATIENT (INCLUDING MEDICARE PART A),COMMERCIAL,CONDITIONS,DISEASE_X,1


In [156]:
#sort the resulted data frame by patient ID to review 
fact_txn_disease_x_features = fact_txn_disease_x_features.sort_values(by='PATIENT_ID',ascending=True)
fact_txn_disease_x_features.head(30)

,TXN_DT,PATIENT_ID,HCP_ID,TXN_LOCATION_TYPE,INSURANCE_TYPE,TXN_TYPE,TXN_DESC,RANK
91556,2022-06-11,1,24633,EMERGENCY ROOM - HOSPITAL,COMMERCIAL,CONDITIONS,DISEASE_X,1
105223,2022-06-01,10,12379,EMERGENCY ROOM - HOSPITAL,COMMERCIAL,CONDITIONS,DISEASE_X,1
79658,2022-06-08,100,3559,OFFICE,COMMERCIAL,CONDITIONS,DISEASE_X,1
57246,2022-06-08,1000,NaN,INDEPENDENT LABORATORY,COMMERCIAL,CONDITIONS,DISEASE_X,1
16495,2022-06-15,1000,NaN,INDEPENDENT LABORATORY,COMMERCIAL,CONDITIONS,DISEASE_X,2
50438,2022-06-10,1001,461,TELEHEALTH PROVIDED OTHER THAN IN PATIENT'S HOME,COMMERCIAL,CONDITIONS,DISEASE_X,1
80693,2022-06-21,1002,NaN,INDEPENDENT LABORATORY,MEDICAID,CONDITIONS,DISEASE_X,1
86346,2022-06-18,1003,17146,OFFICE,COMMERCIAL,CONDITIONS,DISEASE_X,1
111909,2022-06-20,1003,17146,OFFICE,COMMERCIAL,CONDITIONS,DISEASE_X,2
68151,2022-06-10,1004,19500,URGENT CARE FACILITY,COMMERCIAL,CONDITIONS,DISEASE_X,1


In [157]:
diagnosed_patients_count = fact_txn_disease_x_features['PATIENT_ID'].nunique()
diagnosed_patients_count

4020

In [158]:
total_patients_count = fact_txn['PATIENT_ID'].nunique()
total_patients_count

4020

In [159]:
fact_txn_disease_x_features[fact_txn_disease_x_features['PATIENT_ID'] == '683']

,TXN_DT,PATIENT_ID,HCP_ID,TXN_LOCATION_TYPE,INSURANCE_TYPE,TXN_TYPE,TXN_DESC,RANK
1832,2022-06-01,683,16981,UNASSIGNED,MEDICAID,CONDITIONS,DISEASE_X,3
19723,2022-05-01,683,16981,UNASSIGNED,MEDICAID,CONDITIONS,DISEASE_X,2
9199,2022-04-01,683,16981,UNASSIGNED,MEDICAID,CONDITIONS,DISEASE_X,1


In [160]:
#remove rows for ranks > 1 
fact_txn_disease_x_features = fact_txn_disease_x_features.loc[fact_txn_disease_x_features['RANK'] == 1]
fact_txn_disease_x_features.head(30)

,TXN_DT,PATIENT_ID,HCP_ID,TXN_LOCATION_TYPE,INSURANCE_TYPE,TXN_TYPE,TXN_DESC,RANK
91556,2022-06-11,1,24633,EMERGENCY ROOM - HOSPITAL,COMMERCIAL,CONDITIONS,DISEASE_X,1
105223,2022-06-01,10,12379,EMERGENCY ROOM - HOSPITAL,COMMERCIAL,CONDITIONS,DISEASE_X,1
79658,2022-06-08,100,3559,OFFICE,COMMERCIAL,CONDITIONS,DISEASE_X,1
57246,2022-06-08,1000,NaN,INDEPENDENT LABORATORY,COMMERCIAL,CONDITIONS,DISEASE_X,1
50438,2022-06-10,1001,461,TELEHEALTH PROVIDED OTHER THAN IN PATIENT'S HOME,COMMERCIAL,CONDITIONS,DISEASE_X,1
80693,2022-06-21,1002,NaN,INDEPENDENT LABORATORY,MEDICAID,CONDITIONS,DISEASE_X,1
86346,2022-06-18,1003,17146,OFFICE,COMMERCIAL,CONDITIONS,DISEASE_X,1
68151,2022-06-10,1004,19500,URGENT CARE FACILITY,COMMERCIAL,CONDITIONS,DISEASE_X,1
12937,2022-06-21,1005,16863,OFFICE,COMMERCIAL,CONDITIONS,DISEASE_X,1
82947,2022-06-15,1006,19002,EMERGENCY ROOM - HOSPITAL,COMMERCIAL,CONDITIONS,DISEASE_X,1


In [161]:
print(fact_txn_disease_x_features['PATIENT_ID'].nunique())
print(fact_txn_disease_x_features['PATIENT_ID'].count())

4020
4020


In [162]:
fact_txn_highcont = fact_txn.groupby('PATIENT_ID')['TXN_DESC'].apply(lambda x: (x == 'HIGH_CONTRAINDICATION').sum()).reset_index(name='HIGHCONT_COUNT')
fact_txn_highcont.loc[fact_txn_highcont['HIGHCONT_COUNT'] > 1]

,PATIENT_ID,HIGHCONT_COUNT
269,1240,48
444,1399,2
462,1414,5
654,1588,6
680,1610,4
792,1711,24
857,1770,20
893,1802,22
1109,1998,18
1115,2001,5


In [163]:
fact_txn_medcont = fact_txn.groupby('PATIENT_ID')['TXN_DESC'].apply(lambda x: (x == 'MEDIUM_CONTRAINDICATION').sum()).reset_index(name='MEDCONT_COUNT')
fact_txn_medcont.loc[fact_txn_medcont['MEDCONT_COUNT'] > 1]

,PATIENT_ID,MEDCONT_COUNT
8,1005,20
127,1112,11
165,1147,10
237,1211,12
277,1248,9
...,...,...
3928,916,38
3945,931,20
3982,965,15
4003,984,20


In [164]:
fact_txn_lowcont = fact_txn.groupby('PATIENT_ID')['TXN_DESC'].apply(lambda x: (x == 'LOW_CONTRAINDICATION').sum()).reset_index(name='LOWCONT_COUNT')
fact_txn_lowcont.loc[fact_txn_lowcont['LOWCONT_COUNT'] > 1]

,PATIENT_ID,LOWCONT_COUNT
4,1001,3
7,1004,6
9,1006,2
34,1029,8
44,1038,2
...,...,...
3971,955,4
3974,958,3
3986,969,2
4002,983,17


In [165]:
# Define the directory to save the features
features_directory = '../../data/features'

# Ensure the directory exists
os.makedirs(features_directory, exist_ok=True)

# List of dataframes to save
dfs_to_save = {
    'grp_txn_lowcont': fact_txn_lowcont,
    'grp_txn_medcont': fact_txn_medcont,
    'grp_txn_highcont': fact_txn_highcont,
    'grp_txn_num_conditions': fact_txn_num_conditions,
    'grp_txn_num_contraindications': fact_txn_num_contraindications,
    'grp_txn_num_symptoms': fact_txn_num_symptoms,
    'grp_txn_treatment': fact_txn_treatment,
    'dim_patient': dim_patient,
    'dim_physician': dim_physician,
    'grp_txn_disease_x_features': fact_txn_disease_x_features
}

# Save each dataframe as a CSV file
for name, df in dfs_to_save.items():
    file_path = os.path.join(features_directory, f'{name}.csv')
    df.to_csv(file_path, index=False)